In [ ]:
import docplex
from docplex.mp.model import Model
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import DistanceMetric
from math import radians

### Define model

In [ ]:
model = Model (name = "OEM remanu sep. facilities deterministic")

##### Deterministic model only used for calculation of VSS and EVPI

### Definition of sets

In [ ]:
d= [1] #different reprocessing option
c= [1,2,3,4] #sets of collection center
m= [1] #OEM manufacturing
i=[1] #intermediate collection center
t=range (1,61) #time periods in months
w= range(1,601) #workshops
s=[1] #sets of scenarios for yield of returning batteries (beta) (is equal to one in the deterministic setting)
r= [1] #location for remanufacturing facility

### Definition of parameters 

In [ ]:
data = pd.ExcelFile("ExcelImport/Masterarbeit_Input_2.xlsx")

##Fixed cost
#Fixed cost opening intermediate collection center
Fi = data.parse("fixed cost", header=None)[1][0]
#Fixed cost opening remanufacturing facility per month (depreciation)
Fr = data.parse("fixed cost", header=None)[1][1]
#Fixed cost opening collection center for used batteries per month (depreciation)
Fc = data.parse("fixed cost", header=None)[1][2]

##Yields
#rate of return of batteries
alpha = np.tile(np.array(data.parse("yields", header=None)[1][0]), 601,)
#rate of batteries going to reman instead of Re-X.
delta= np.tile(np.array(data.parse("yields", header=None)[1][2]), 601,)
#yield of remanufacturing
gamma = np.tile(np.array(data.parse("yields", header=None)[1][3]), 601,)

## Change manually depending on calculation
#### Expected value of beta: 0.0415
#rate of severly damaged batteries going directly and alone to ReX (scenarios)
#beta = np.tile([0.1, 0.04, 0.01], (601,61,1)) (stochastic setting)
beta = np.tile([0.04], (601,61,1)) #(deterministic setting)
#probability for each scenario (1= pessimistic, 2=normal, 3=optimisitic) (in deterministic setting: all is equal to 1)
prob = [1]

##Capacities
#Collection point used battery
capc= data.parse("capa", header=None)[1][0]
#Intermediate intermediate collection points new battery
capi= data.parse("capa", header=None)[1][1]
#Capacity at each workshop for storage
capw= data.parse("capa", header=None)[1][2]
#Capacity transportation CD
captcd = data.parse("capa", header=None)[1][3]
captcdmin = data.parse("capa", header=None)[1][6]
#Capacity manufacturing operation
capm = data.parse("capa", header=None)[1][4]
#Capacity remanufacturing operation
capr = data.parse("capa", header=None)[1][5]

##Variable Costs
#Variable cost of manufacturing new battery
MC= data.parse("var cost", header=None)[1][0]
#Variable cost of remanufacturing battery in 2nd facility (is cheaper than in hybrid)
RC= data.parse("var cost", header=None)[1][1]
#Variable cost of collection center
UC= data.parse("var cost", header=None)[1][3]
#Variable cost of workshop
WC = data.parse("var cost", header=None)[1][4]
#Variable cost of intermediate collection center
IC = data.parse("var cost", header=None)[1][5]
#Variable cost of Re-X strategy (different reprocessing strategies)
RXC = data.parse("var cost", header=None)[1][6]
#setup cost for manufacturing in period t
MS = data.parse("var cost", header=None)[1][7]
#setup cost for remanufacturing in period t
MR  = data.parse("var cost", header=None)[1][8]
#setup cost transportation C-D
SCTCD = data.parse("var cost", header=None)[1][9]
#handling cost workshop
HW = data.parse("var cost", header=None)[1][10]
#Savings if manufacturing and remanufacturing at the same time
SMR = data.parse("var cost", header=None)[1][11]

##Transportation Costs per battery per km
#Used batteries
TCu = data.parse("transportation cost", header=None)[1][0]
#Damaged batteries
TCd = data.parse("transportation cost", header=None)[1][1]
#New batteries to intermediate
TCni = data.parse("transportation cost", header=None)[1][2]
#New batteries to workshop
TCnw = data.parse("transportation cost", header=None)[1][3]

###Revenue
#New batteries
Pn = data.parse("revenue", header=None)[1][0]
#Remanufactured batteries
Pr = data.parse("revenue", header=None)[1][1]

In [ ]:
# Deterministic demand function new
intial_demand=4800

demand=np.matrix([[1]])
for q in range(59):
    demand=np.r_[demand,[[(1+0.008)**(q+1)]]]
demand=demand*intial_demand/600    
demand=np.ceil(demand)

Dn=np.full([60,601], demand)

# Deterministic demand function remanufactured
intial_demand=400

demand=np.matrix([[1]])
for q in range(59):
    demand=np.r_[demand,[[(1+0.025)**(q+1)]]]
demand=demand*intial_demand/600    
demand=np.ceil(demand)

Dr=np.full([60,601], demand)

### Definition of decision variables

In [ ]:
## Network opening decision variables are decisions on the first stage (concerns Ac, Bi and Br)
## Second stage decisions include the index s for the different scenarios


## As docplex only allows direct adding of variables up to 3 dimensions, flows have to be indexed like this
Xwc = model.continuous_var_dict((w1, c1, t1, s1)
                                for w1 in w
                                for c1 in c
                                for t1 in t
                                for s1 in s) # old product flowing from w to c in t 
Xwd = model.continuous_var_dict((w2, d2, t2, s2)
                                for w2 in w
                                for d2 in d
                                for t2 in t
                                for s2 in s)# old product flowing from w to d in t 
Xcd = model.continuous_var_dict((c3, d3, t3, s3)
                                for c3 in c
                                for d3 in d
                                for t3 in t
                                for s3 in s)# old product flowing from c to d in t
Ymi = model.continuous_var_dict((m4, i4, t4, s4)
                                for m4 in m
                                for i4 in i
                                for t4 in t
                                for s4 in s)# new product flowing from m to i in t 
Yiw = model.continuous_var_dict((i5, w5, t5, s5)
                                for i5 in i
                                for w5 in w
                                for t5 in t
                                for s5 in s) # new product flowing from i to w in t 
Xcr = model.continuous_var_dict((c6, r6, t6, s6)
                                for c6 in c 
                                for r6 in r
                                for t6 in t
                                for s6 in s)# old product flowing from c to m in t
Xri = model.continuous_var_dict((r7, i7, t7, s7)
                                for r7 in r
                                for i7 in i
                                for t7 in t
                                for s7 in s) # remanufactured product flowing from m to i in t
Xiw = model.continuous_var_dict((i8, w8, t8, s8)
                                for i8 in i
                                for w8 in w
                                for t8 in t
                                for s8 in s) # remanufactured product flowing from i to w in t
Xwdrem = model.continuous_var_dict((w9, d9, t9, s9)
                                   for w9 in w
                                   for d9 in d
                                   for t9 in t
                                   for s9 in s) # remanufactured product flowing from w to d in t 

Lin = model.continuous_var_cube(i, t, s, name = "invIn") # inventory level intermediate in t new batteries
Lir= model.continuous_var_cube(i, t, s, name = "invIr") # inventory level reman. batteries intermediate in t 
Lc = model.continuous_var_cube(c, t, s, name = "invC") # inventory level collection used products in t 
Lnw = model.continuous_var_cube(w, t, s, name = "invWn") # inventory level new batteries at workshop in t 
Luw = model.continuous_var_cube(w, t, s, name = "invWu") # inventory level used batteries at workshop in t 

LnwR = model.continuous_var_cube(w, t, s, name = "invWnRem") # inventory level new batteries at workshop in t rem
LuwR = model.continuous_var_cube(w, t, s, name = "invWuRem") # inventory level used batteries at workshop in t rem

Ac = model.binary_var_matrix(c, t, name = "setupC")  #Binary for collection center used products
Bi = model.binary_var_matrix(i, t, name = "setupI")  #Binary for intermediate collect. new products
Cw = model.binary_var_cube(w, t, s, name = "setupW")  #Binary for workshop storage
Sm = model.binary_var_cube(m, t, s, name = "setupM")  #Binary for manufacturing setup
Sr = model.binary_var_cube(r, t, s, name = "setupR")  #Binary for remanufacturing setup
Br = model.binary_var_matrix(r, t, name = "buildR")  #Binary for building reman. facility


Bin = model.binary_var_cube(w, t, s, name = "Binary demand")  #Binary for demand condition

Subst = model.binary_var_cube(w, t, s, name = "Substituted products") # Substituted demand for remanufactured with new batteries

Stcd = model.binary_var_cube(c, t, s, name = "setupTranspCD")  #Binary for setup renting transporation C-D

SalesN = model.continuous_var_cube(w, t, s, name = "SalesN") # Sales new batteries at workshop in t 
SalesR = model.continuous_var_cube(w, t, s, name = "SalesR") # Sales remanufactured batteries at workshop in t 


### Definition of constraints

In [ ]:
##Demand first 12 periods to initialize model
#(demand refers to T-1, as python starts counting from zero, not one// Model meets demand from current period, not from previous period)
for W in w:
    for T in range (1,13):
        for S in s:
            model.add_constraint(SalesN[W,T,S] <= Dr[T-1,W-1] + Dn[T-1,W-1])
            model.add_constraint(SalesN[W,T,S] >= Dr[T-1,W-1])
            model.add_constraint(SalesR[W,T,S]==0)
model.add_constraints(Subst[W,T,S]==Dr[T-1,W-1] for W in w for T in range(1,13) for S in s)

###Sales remanufactured batteries

##Introduction of binary variable, to indicate when there are too few remanufactured batteries available to meet the respective demand
## If binary variable =0 -> need for additional new batteries
## If binary variable =1 -> no need for additional new batteries, as there are sufficient remanufactured batteries available

## Essentially: SalesR <= Min{Xiw ; Dr}
## SalesN <= Max{Dn+Dr-Xiw; Dn}
## See model formulation in thesis 

for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(Dr[T-1,W-1]-model.sum(Xiw[I,W,T,S] for I in i) <= 9999999*(1-Bin[W,T,S]))
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(model.sum(Xiw[I,W,T,S] for I in i)-Dr[T-1,W-1] <= 9999999*Bin[W,T,S])
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(SalesR[W,T,S] <=model.sum(Xiw[I,W,T,S] for I in i))
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(SalesR[W,T,S] <=Dr[T-1,W-1]) 
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(SalesR[W,T,S] >=Dr[T-1,W-1]-9999999*(1-Bin[W,T,S]))
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(SalesR[W,T,S] >=model.sum(Xiw[I,W,T,S] for I in i)-9999999*(Bin[W,T,S]))
        
##Sales new batteries
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(SalesN[W,T,S] <= Dn[T-1,W-1]+9999999*(1-Bin[W,T,S])) 
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(SalesN[W,T,S] <= Dn[T-1,W-1] + Dr[T-1,W-1]-model.sum(Xiw[I,W,T,S] for I in i)+9999999*(Bin[W,T,S]))       

In [ ]:
## penalty for demand substitution 
## activated in case there are too few remanufactured batteries available, indicated by binary variable from step before

for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(Subst[W,T,S] >= Dr[T-1,W-1]-SalesR[W,T,S]-9999999*(Bin[W,T,S])) 
        
for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(Subst[W,T,S] >= 0-9999999*(1-Bin[W,T,S]))

for T in range (13,61):
    for W in w:
        for S in s:
            model.add_constraint(Subst[W,T,S] <= Dr[T-1,W-1])         

In [ ]:
#### Quality of used batteries
for W in w:  
    for T in range (13,61):
        for S in s:
            model.add_constraint((alpha[W]*beta[W,T,S-1]*SalesN[W,T-12,S]) <= model.sum(Xwd[W,D,T,S] for D in d))

#Capacity constraint used battery collection in collection center C
model.add_constraints(Lc[C,T,S] <= capc*Ac[C,T] for C in c for T in t for S in s)

#Capacity constraint intermediate, new battery collection
model.add_constraints((Lin[I,T,S]+ Lir[I,T,S]) <= capi*Bi[I,T] for S in s for I in i for T in t)

#Capacity constraint workshop for storage
model.add_constraints((Luw[W,T,S]+Lnw[W,T,S]+LuwR[W,T,S]+LnwR[W,T,S]) <= capw*Cw[W,T,S] for S in s for W in w for T in t)

#Capacity constraint for transportation CD and minimum for batch-transportation
model.add_constraints(model.sum(Xcd[C,D,T,S] for D in d) <= captcd*Stcd[C,T,S] for T in t for C in c for S in s)
model.add_constraints(model.sum(Xcd[C,D,T,S] for D in d) >= captcdmin*Stcd[C,T,S] for T in t for C in c for S in s)

#Capacity constraint manufacturing
model.add_constraints(model.sum(Ymi[M,I,T,S] for I in i) <= capm*Sm[M,T,S] for S in s for M in m for T in t)

#Capacity constraint remanufacturing
model.add_constraints(model.sum(Xri[R,I,T,S] for I in i) <= capr*Sr[R,T,S] for S in s for R in r for T in t)

#Opening collection center
for C in c:
    for T in t:
        for S in s:
            model.add_constraint(model.sum(Xwc[W,C,T,S]  for W in w) <= 9999999*Ac[C,T])

#Open max. 4 collection center out of the 4 possible locations (first stage decision)
for T in t:
    model.add_constraint(model.sum(Ac[C,T] for C in c)<=4)
    
#Not closing constraint if opened facility (first two constraints are first stage decisions)
model.add_constraints(Ac[C,T]<=Ac[C,T+1] for C in c for T in range (1,60))
model.add_constraints(Bi[I,T]<=Bi[I,T+1] for I in i for T in range (1,60))
model.add_constraints(Cw[W,T,S]<=Cw[W,T+1,S] for S in s for W in w for T in range (1,60))
model.add_constraints(Sr[R,T,S]<=Sr[R,T+1,S] for S in s for R in r for T in range (1,60))

#Enforce opening of facility (workshops always, collection center and intermediate at least once) (first two constraints are first stage decisions)
model.add_constraint(model.sum(Ac[C,T] for C in c for T in t)>= 1 )
model.add_constraint(model.sum(Bi[I,T] for I in i for T in t)>= 1 )
for W in w:
    for T in t:
        for S in s:
            model.add_constraint(Cw[W,T,S]>= 1) 
model.add_constraint(model.sum(Sr[R,T,S] for S in s for R in r for T in range (1,61))>= 1 )

###Inventory Balance for periods 2 to 60
#for collection-center "C"
model.add_constraints(model.sum(Xwc[W,C,T,S] for W in w) + Lc[C,T-1,S] -model.sum(Xcd[C,D,T,S] for D in d) - model.sum(Xcr[C,R,T,S] for R in r) == Lc[C,T,S] for C in c for T in range (2,61) for S in s)

#for intermediate "I" (first for remanufactured, then for new batteries)
model.add_constraints(model.sum(Xri[R,I,T,S] for R in r)+ Lir[I,T-1,S] -model.sum(Xiw[I,W,T,S] for W in w)== Lir[I,T,S] for I in i for T in range (2,61) for S in s)
model.add_constraints(model.sum(Ymi[M,I,T,S] for M in m)+ Lin[I,T-1,S] -model.sum(Yiw[I,W,T,S] for W in w)== Lin[I,T,S] for I in i for T in range (2,61) for S in s)

#for Workshops (1. New batteries inflow, 2. remanufactured batteries inflow, 3. outflow used batteries, 4. outflow used remanufactured batteries) )
model.add_constraints(model.sum(Yiw[I,W,T,S] for I in i) + Lnw[W,T-1,S] - SalesN[W,T,S] == Lnw[W,T,S] for S in s for W in w for T in range (2,61))
model.add_constraints(model.sum(Xiw[I,W,T,S] for I in i) + LnwR[W,T-1,S] - SalesR[W,T,S] == LnwR[W,T,S] for S in s for W in w for T in range (2,61))
model.add_constraints(alpha[W]*(SalesN[W,T-12,S] ) + Luw[W,T-1,S] - model.sum(Xwc[W,C,T,S] for C in c)- model.sum(Xwd[W,D,T,S] for D in d) == Luw[W,T,S] for S in s for W in w for T in range (13,61))
model.add_constraints(alpha[W]*(SalesR[W,T-12,S] ) + LuwR[W,T-1,S] - model.sum(Xwdrem[W,D,T,S] for D in d)  == LuwR[W,T,S] for S in s for W in w for T in range (13,61))
model.add_constraint(model.sum(Xwc[W,C,T,S] for S in s for C in c for T in range (1,13) for W in w)==0)

#Initial inventory level at period 1 collection point
for C in c:
    for S in s:
        model.add_constraint(Lc[C,1,S]==0)   
    
#Initial inventory level at period 1 intermediate new and remanufactured    
for I in i:
    for S in s:
        model.add_constraint(model.sum(Ymi[M,I,1,S] for M in m)+ 0 -model.sum(Yiw[I,W,1,S] for W in w)== Lin[I,1,S] )
for I in i:
    for T in range (1,14):
        for S in s:
            model.add_constraint(Lir[I,T,S]==0)
            
#Initial inventory level at period 1 workshop for all 4 separated inventory levels    
for W in w:
    for S in s:
        model.add_constraint(model.sum(Yiw[I,W,1,S] for I in i)  + 10 - SalesN[W,1,S] == Lnw[W,1,S])                
for W in w:
    for T in range (1,13):
        for S in s:
            model.add_constraint(Luw[W,T,S]==0)
for W in w:
    for T in range (1,26):
        for S in s:
            model.add_constraint(LuwR[W,T,S]==0) 
for W in w:
    for T in range (1,14):
        for S in s:
            model.add_constraint(LnwR[W,T,S]==0)         


In [ ]:
#Additional for remanufacturing

#yield of products going to remanufacturing instead of Re-X        
for C in c:
    for T in t: 
        for S in s:
            model.add_constraint(delta[C]*model.sum(Xwc[W,C,T,S] for W in w) == model.sum(Xcr[C,R,T,S] for R in r))

#Yield of remanufacturing  
for R in r:
    for T in range(1,60):
        for S in s:
            model.add_constraint(gamma[R]*(model.sum(Xcr[C,R,T,S] for C in c))== model.sum(Xri[R,I,T+1,S] for I in i))
            
#Build remanufacturing facility (first stage decisions)
model.add_constraints(Br[R,T]>= Sr[R,T,S] for R in r for T in t for S in s)

In [ ]:
#Non negativity
model.add_constraints(Xwc[W,C,T,S]>= 0 for W in w for T in t for C in c for S in s)
model.add_constraints(Xcr[C,R,T,S]>= 0 for R in r for T in t for C in c for S in s)
model.add_constraints(Xwd[W,D,T,S]>= 0 for W in w for T in t for D in d for S in s)
model.add_constraints(Xcd[C,D,T,S]>= 0 for C in c for T in t for D in d for S in s)
model.add_constraints(Xri[R,I,T,S]>= 0 for R in r for T in t for I in i for S in s)
model.add_constraints(Xiw[I,W,T,S]>= 0 for W in w for T in t for I in i for S in s)
model.add_constraints(Ymi[M,I,T,S]>= 0 for M in m for T in t for I in i for S in s)
model.add_constraints(Yiw[I,W,T,S]>= 0 for W in w for T in t for I in i for S in s) 
model.add_constraints(Xwdrem[W,D,T,S]>= 0 for W in w for T in t for D in d for S in s)

model.add_constraints(Lc[C,T,S]>= 0 for T in t for C in c for S in s)
model.add_constraints(Luw[W,T,S]>= 0 for T in t for W in w for S in s)
model.add_constraints(Lnw[W,T,S]>= 0 for T in t for W in w for S in s)

model.add_constraints(LuwR[W,T,S]>= 0 for T in t for W in w for S in s)
model.add_constraints(LnwR[W,T,S]>= 0 for T in t for W in w for S in s)

model.add_constraints(Lir[I,T,S]>= 0 for T in t for I in i for S in s)
model.add_constraints(Lin[I,T,S]>= 0 for T in t for I in i for S in s)

model.add_constraints(Subst[W,T,S]>= 0 for T in t for W in w for S in s)

### Definition Objective Function

In [ ]:
#Revenue minus penalty for demand substitution
REV = Pn*model.sum(prob[S-1]*SalesN[W,T,S] for W in w for T in t for S in s) + Pr*model.sum(prob[S-1]*SalesR[W,T,S] for W in w for T in t for S in s)-(Pn-Pr)*(model.sum(prob[S-1]*Subst[W,T,S] for W in w for T in t for S in s))

In [ ]:
#Fixed opening cost (depreciated, per month)  (collection center, intermediate, remanufacturing)
FOCC= Fc*model.sum(Ac[C,T] for C in c for T in t)  
FOCI=Fi*model.sum(Bi[I,T] for I in i for T in t) 
FOCRM=Fr*model.sum(Br[R,T] for R in r for T in t)
#Setup cost manufacturing
SCM = MS*model.sum(prob[S-1]*Sm[M,T,S] for S in s for M in m for T in t)
#Setup cost remanufacturing
SCR = MR*model.sum(prob[S-1]*Sr[R,T,S] for S in s for R in r for T in t)
#Setup cost transportation CD
SCTCDU = SCTCD*model.sum(prob[S-1]*Stcd[C,T,S] for C in c for T in t for S in s)

In [ ]:
##Only activate, in case of new sampling for workshops

#DF_3=DF_3.sample(n=600)
#DF_3.reset_index(drop=True, inplace=True)
#DF_3.to_csv("workshop_sample.csv")
DF_3=pd.read_csv("workshop_sample.csv")

In [ ]:
#transportation cost new batteries to intermediate
TRCOSTN= TCni*DF_3.loc[1,"dmi"]*(model.sum(prob[S-1]*Ymi[M,I,T,S] for M in m for I in i for T in t for S in s))

In [ ]:
#transportation cost new batteries to workshop (first (TRCOSTNW) for new, (TRCOSTRW) for remanufactured, that can be transported as new)
TCnw_matrix=TCnw*DF_3.loc[:,"diw"]
TRCOSTNW= model.sum(prob[S-1]*(Yiw[I,W,T,S])*TCnw_matrix[W-1] for S in s for I in i for T in t for W in w)
TRCOSTRW= model.sum(prob[S-1]*(Xiw[I,W,T,S])*TCnw_matrix[W-1] for S in s for I in i for T in t for W in w)

In [ ]:
#transportation cost batteries R1->I (R1 was chosen as the optimal location out of 4 locations in the deterministic setting)
TRI_matrix=TCni*DF_3.loc[:,"dr1i"]
TRCOSTRI=model.sum(prob[S-1]*Xri[1,I,T,S]*TRI_matrix[I] for S in s for I in i for T in t)

In [ ]:
#transportation cost used batteries from workshops to C1
TCU_matrix=TCu*DF_3.loc[:,"dwc"]
TRCOSTU1=model.sum(prob[S-1]*Xwc[W,1,T,S]*TCU_matrix[W-1]  for T in t for W in w for S in s)

#transportation cost used batteries from workshops to C2
TCU_matrix2=TCu*(DF_3.loc[:,"dwc2"])
TRCOSTU2=model.sum(prob[S-1]*Xwc[W,2,T,S]*TCU_matrix2[W-1] for T in t for W in w for S in s)

#transportation cost used batteries from workshops to C3
TCU_matrix3=TCu*(DF_3.loc[:,"dwc3"])
TRCOSTU3=model.sum(prob[S-1]*Xwc[W,3,T,S]*TCU_matrix3[W-1] for T in t for W in w for S in s)

#transportation cost used batteries from workshops to C4
TCU_matrix4=TCu*(DF_3.loc[:,"dwc4"])
TRCOSTU4=model.sum(prob[S-1]*Xwc[W,4,T,S]*TCU_matrix4[W-1] for T in t for W in w for S in s)

TRCOSTU =TRCOSTU1 + TRCOSTU2 + TRCOSTU3 + TRCOSTU4

In [ ]:
#transportation cost damaged batteries from workshop
TCd_matrix=TCd*DF_3.loc[:,"dwd"]
TRCOSTWD = model.sum(prob[S-1]*(Xwd[W,D,T,S]+Xwdrem[W,D,T,S])*TCd_matrix[W-1] for D in d for T in t for W in w for S in s)

In [ ]:
#transportation cost damaged batteries from collection center 1
TRCOSTCD1= TCd*DF_3.loc[1,"dcd"] *model.sum(prob[S-1]*Xcd[1,D,T,S] for S in s for D in d for T in t)

#transportation cost damaged batteries from collection center 2
TRCOSTCD2= TCd*DF_3.loc[1,"dc2d"] *model.sum(prob[S-1]*Xcd[2,D,T,S] for S in s for D in d for T in t)

#transportation cost damaged batteries from collection center 3
TRCOSTCD3= TCd*DF_3.loc[1,"dc3d"] *model.sum(prob[S-1]*Xcd[3,D,T,S] for S in s for D in d for T in t)

#transportation cost damaged batteries from collection center 4
TRCOSTCD4= TCd*DF_3.loc[1,"dc4d"] *model.sum(prob[S-1]*Xcd[4,D,T,S] for S in s for D in d for T in t)

TRCOSTCD=TRCOSTCD1 + TRCOSTCD2 + TRCOSTCD3 + TRCOSTCD4

In [ ]:

#transportation cost used batteries from collection center 1 to Remanufacturer
TRCOSTUREM1 = TCu*DF_3.loc[1,"dcr1"]*(model.sum(prob[S-1]*Xcr[1,1,T,S] for S in s for T in t))
 
#transportation cost used batteries from collection center 2 to Remanufacturer
TRCOSTUREM2 = TCu*DF_3.loc[1,"dc2r1"]*(model.sum(prob[S-1]*Xcr[2,1,T,S] for S in s for T in t))

#transportation cost used batteries from collection center 3 to Remanufacturer
TRCOSTUREM3 = TCu*DF_3.loc[1,"dc3r1"]*(model.sum(prob[S-1]*Xcr[3,1,T,S] for S in s for T in t))

#transportation cost used batteries from collection center 4 to Remanufacturer
TRCOSTUREM4 = TCu*DF_3.loc[1,"dc4r1"]*(model.sum(prob[S-1]*Xcr[4,1,T,S] for S in s for T in t))

TRCOSTUREM = TRCOSTUREM1 + TRCOSTUREM2 + TRCOSTUREM3 + TRCOSTUREM4

In [ ]:
#cost of storage in workshop (1. used batteries outflow, 2. new batteries inflow, 3. used remanufactured batteries outflow, 4. remanufactured batteries inflow)
CWS_U_OUT = WC*(model.sum(prob[S-1]*Luw[W,T,S] for W in w for T in t for S in s))
CWS_N_IN = WC*(model.sum(prob[S-1]*Lnw[W,T,S] for S in s for W in w for T in t)) 
CWS_UR_OUT = WC*(model.sum(prob[S-1]*LuwR[W,T,S] for S in s for W in w for T in t)) 
CWS_R_IN = WC*(model.sum(prob[S-1]*LnwR[W,T,S] for S in s for W in w for T in t))

In [ ]:
#Cost for Re-X (1. for damaged batteries directly from W, 2. for damaged batteries from C, 3. for batteries coming back after being already remanufactured) )
REXC_WD = RXC*(model.sum(prob[S-1]*Xwd[W,D,T,S] for S in s for W in w for D in d for T in t)) 
REXC_CD = RXC*(model.sum(prob[S-1]*Xcd[C,D,T,S] for S in s for C in c for D in d for T in t)) 
REXC_WD_rem = RXC*(model.sum(prob[S-1]*Xwdrem[W,D,T,S] for S in s for W in w for D in d for T in t))

In [ ]:
#variable cost (manufacturing, intermediate storage remanufactured, intermediate storage new, collection center, remanufacturing)
VARM = MC*(model.sum(prob[S-1]*Ymi[M,I,T,S] for S in s for M in m for I in i for T in t)) 
VARI_R = IC*(model.sum(prob[S-1]*Lir[I,T,S] for I in i for T in t for S in s))
VARI_N = IC*(model.sum(prob[S-1]*Lin[I,T,S] for S in s for I in i for T in t))
VARC = UC*(model.sum(prob[S-1]*Lc[C,T,S] for C in c for T in t for S in s)) 
VARRM = RC*(model.sum(prob[S-1]*Xri[R,I,T,S] for R in r for I in i for T in t for S in s))

In [ ]:
#Handling cost in workshop for exchanges of battery
HanW = HW*model.sum(prob[S-1]*Xwd[W,D,T,S] for S in s for W in w for D in d for T in t) + HW*model.sum(prob[S-1]*Xwc[W,C,T,S] for S in s for W in w for C in c for T in t) + HW*model.sum(prob[S-1]*Xwdrem[W,D,T,S] for S in s for W in w for D in d for T in t)

In [ ]:
#Profit Calculation
P=REV- (HanW + FOCC+ FOCRM +FOCI + TRCOSTN + TRCOSTNW + TRCOSTRW + TRCOSTU + TRCOSTCD + TRCOSTWD + TRCOSTUREM + CWS_U_OUT + CWS_N_IN + CWS_UR_OUT + CWS_R_IN + VARM + VARI_R + VARI_N + VARC + VARRM + REXC_WD + REXC_CD + REXC_WD_rem +SCM + SCR + SCTCDU + TRCOSTRI)

In [ ]:
model.parameters.mip.tolerances.mipgap = 0.001
model.maximize(P)
model.solve()

print(model.solve_details)

Output generation for Excel-Export

In [ ]:
##Definition of new decision variables


YiwNew = model.integer_var_matrix(t, s, name = "agg. flow I-W") #agg. flow I-W"  over W in T
XwcNew = model.integer_var_cube(c, t, s, name = "agg. flow W-C") #agg. flow W-C"  over C in T
XwdNew = model.integer_var_matrix(t, s, name = "agg. flow W-D") #agg. flow W-D"  over D in T
XcdNew = model.integer_var_cube(c, t, s, name = "agg. flow C-D") #agg. flow C-D"  over C in T
YmiNew = model.integer_var_matrix(t, s, name = "agg. flow M-I") #agg. flow M-I"  over I in T
XcrNew = model.integer_var_cube(c, t, s, name = "agg. flow C-R") #agg. flow C-R"  over C in T
XriNew = model.integer_var_matrix(t, s, name = "agg. flow R-I rem") #agg. flow R-I"  over R in T
XiwNew = model.integer_var_matrix(t, s, name = "agg. flow I-W rem") #agg. flow I-W"  over I in T

SalesNnew = model.integer_var_matrix(t, s, name = "agg. SalesN") #agg. Sales new batteries over W
SalesRnew = model.integer_var_matrix(t, s, name = "agg. SalesR") #agg. Sales rem. batteries over W
XremwdNew = model.integer_var_matrix(t, s, name = "Rem agg. flow W-D") #agg. flow W-D"  over D in T, time constraint for returns

LnwNew = model.integer_var_matrix(t, s, name = "agg. Inw N W") #agg. inventory over all w of new batteries 
LuwNew = model.integer_var_matrix(t, s, name = "agg. Inw U W") #agg. inventory over all w of used batteries
LnwRNew = model.integer_var_matrix(t, s, name = "agg. Inw R W") #agg. inventory over all w of remanufactured batteries inflow
LuwRNew = model.integer_var_matrix(t, s, name = "agg. Inw RU W") #agg. inventory over all w of remanufactured batteries outflow

LinNew = model.integer_var_matrix(t, s, name = "agg. Inw N I") #agg. inventory over all i of new batteries
LirNew = model.integer_var_matrix(t, s, name = "agg. Inw R I") #agg. inventory over all i of remanu batteries

SubstNew = model.integer_var_matrix(t, s, name = "agg. substituted products") #agg.substituted products

DemNNew = model.integer_var_dict(t, name = "agg. Demand New") #agg. demand new batteries
DemRNew = model.integer_var_dict(t, name = "agg. Demand Rem") #agg. demand rem batteries

SmNew = model.integer_var_matrix(t, s, name = "agg. setup manu") #setup manufacturing
SrNew = model.integer_var_matrix(t, s, name = "agg. setup remanu") #setup remanufacturing
StcdNew = model.integer_var_cube(c, t, s,  name = "add setup transportation") #agg. transportation setup over all scenarios
CwNew = model.integer_var_matrix(t, s,  name = "agg. setup w") #agg. setup W over all scenarios


In [ ]:
## Aggregation of results for excel-export


for T in t:
    for S in s:
        YiwNew[T,S] = model.sum(Yiw[I,W,T,S] for W in w for I in i)
    
for T in t:
    for S in s:
        YmiNew[T,S] = model.sum(Ymi[M,I,T,S] for M in m for I in i)       

for T in t:
    for C in c:
        for S in s:
            XwcNew[C,T,S] = model.sum(Xwc[W,C,T,S] for W in w)

for T in t:
    for S in s:
        XwdNew[T,S] = model.sum(Xwd[W,D,T,S] for D in d for W in w)
    
for T in t:
    for C in c:
        for S in s:
            XcdNew[C,T,S] = model.sum(Xcd[C,D,T,S] for D in d)
for T in t:
    for C in c:
        for S in s:
            XcrNew[C,T,S] = model.sum(Xcr[C,R,T,S] for R in r)

for T in t:
    for S in s:
        XriNew[T,S] = model.sum(Xri[R,I,T,S] for R in r for I in i)
    
for T in t:
    for S in s:
        XiwNew[T,S] = model.sum(Xiw[I,W,T,S] for I in i for W in w)
    
for T in t:
    for S in s:
        XremwdNew[T,S] = model.sum(Xwdrem[W,D,T,S] for D in d for W in w)
for T in t:
    for S in s:
        SalesNnew[T,S] = model.sum(SalesN[W,T,S] for W in w)
for T in t:
    for S in s:
        SalesRnew[T,S] = model.sum(SalesR[W,T,S] for W in w)

for T in t:
    for S in s:
        LnwNew[T,S] = model.sum(Lnw[W,T,S] for W in w)
for T in t:
    for S in s:
        LuwNew[T,S] = model.sum(Luw[W,T,S] for W in w)
    
for T in t:
    for S in s:
        LnwRNew[T,S] = model.sum(LnwR[W,T,S] for W in w)
for T in t:
    for S in s:
        LuwRNew[T,S] = model.sum(LuwR[W,T,S] for W in w)
for T in t:
    for S in s:
        LinNew[T,S] = model.sum(Lin[I,T,S] for I in i)    
for T in t:
    for S in s:
        LirNew[T,S] = model.sum(Lir[I,T,S] for I in i)    
for T in t:
    for S in s:
        SmNew[T,S] = model.sum(Sm[M,T,S] for M in m)
for T in t:
    for S in s:
        SrNew[T,S] = model.sum(Sr[R,T,S] for R in r)
for T in t:
    for S in s:
        CwNew[T,S] = model.sum(Cw[W,T,S] for W in w)

for T in t:
    for S in s:
        SubstNew[T,S] = model.sum(Subst[W,T,S] for W in w)

#Dataframe for deterministic case  

header = ['Period', 'Flow M-I new' , 'Flow I-W new' , 'Flow W-C1', 'Flow W-C2', 'Flow W-C3', 'Flow W-C4', 'Flow W-D', 'Flow C-D1', 'Flow C-D2', 'Flow C-D3', 'Flow C-D4', 'Flow C-R1', 'Flow C-R2', 'Flow C-R3', 'Flow C-R4', 'Flow R-I rem1' ,  'Flow I-W rem',  'Flow W-D rem','Inv Ir','Inv In', 'InvC1', 'InvC2', 'InvC3', 'InvC4', 'InvWn', 'InvWu', 'InvWnR' , 'InvWuR', 'SetupM', 'SetupR', 'SetupTransportCD1', 'SetupTransportCD2', 'SetupTransportCD3', 'SetupTransportCD4', 'setupC1', 'setupC2', 'setupC3', 'setupC4', 'setupI' ,'setupW', 'SalesN', 'SalesR', 'Subst', 'BuildR']

df = pd.DataFrame(columns = header)
for T in t:
    df.loc[len(df), 'Period'] = T
    df.loc[len(df)-1, 'Flow M-I new'] = YmiNew[T,1].solution_value
    df.loc[len(df)-1, 'Flow I-W new'] = YiwNew[T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C1'] = XwcNew[1,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C2'] = XwcNew[2,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C3'] = XwcNew[3,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C4'] = XwcNew[4,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-D'] = XwdNew[T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D1'] = XcdNew[1,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D2'] = XcdNew[2,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D3'] = XcdNew[3,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D4'] = XcdNew[4,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-R1'] = XcrNew[1,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-R2'] = XcrNew[2,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-R3'] = XcrNew[3,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-R4'] = XcrNew[4,T,1].solution_value
    df.loc[len(df)-1, 'Flow R-I rem1'] = XriNew[T,1].solution_value   
    df.loc[len(df)-1, 'Flow I-W rem'] = XiwNew[T,1].solution_value
    df.loc[len(df)-1, 'Flow W-D rem'] = XremwdNew[T,1].solution_value
    
    df.loc[len(df)-1, 'Inv Ir'] = LirNew[T,1].solution_value
    df.loc[len(df)-1, 'Inv In'] = LinNew[T,1].solution_value
    df.loc[len(df)-1, 'InvC1'] = Lc[1,T,1].solution_value
    df.loc[len(df)-1, 'InvC2'] = Lc[2,T,1].solution_value
    df.loc[len(df)-1, 'InvC3'] = Lc[3,T,1].solution_value
    df.loc[len(df)-1, 'InvC4'] = Lc[4,T,1].solution_value
    df.loc[len(df)-1, 'InvWn'] = LnwNew[T,1].solution_value
    df.loc[len(df)-1, 'InvWu'] = LuwNew[T,1].solution_value
    df.loc[len(df)-1, 'InvWnR'] = LnwRNew[T,1].solution_value
    df.loc[len(df)-1, 'InvWuR'] = LuwRNew[T,1].solution_value
    df.loc[len(df)-1, 'SetupM'] = SmNew[T,1].solution_value
    df.loc[len(df)-1, 'SetupR'] = SrNew[T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD1'] = Stcd[1,T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD2'] = Stcd[2,T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD3'] = Stcd[3,T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD4'] = Stcd[4,T,1].solution_value
    df.loc[len(df)-1, 'setupC1'] = Ac[1,T].solution_value
    df.loc[len(df)-1, 'setupC2'] = Ac[2,T].solution_value
    df.loc[len(df)-1, 'setupC3'] = Ac[3,T].solution_value
    df.loc[len(df)-1, 'setupC4'] = Ac[4,T].solution_value
    df.loc[len(df)-1, 'setupI'] = Bi[I,T].solution_value
    df.loc[len(df)-1, 'setupW'] = CwNew[T,1].solution_value
    df.loc[len(df)-1, 'SalesN'] = SalesNnew[T,1].solution_value
    df.loc[len(df)-1, 'SalesR'] = SalesRnew[T,1].solution_value
    df.loc[len(df)-1, 'Subst'] = SubstNew[T,1].solution_value 
    df.loc[len(df)-1, 'BuildR'] = Br[1,T].solution_value
print(df)
print("The objective value is ", model.objective_value)
print('REV value is:', REV.solution_value)
print('FOCC value is:', FOCC.solution_value)
print('FOCI value is:', FOCI.solution_value)
print('FOCRM value is:', FOCRM.solution_value)
print('TRCOSTN value is:', TRCOSTN.solution_value)
print('TRCOSTNW value is:', TRCOSTNW.solution_value)
print('TRCOSTRW value is:', TRCOSTRW.solution_value)
print('TRCOSTU value is:', TRCOSTU.solution_value)
print('TRCOSTCD value is:', TRCOSTCD.solution_value)
print('TRCOSTWD value is:', TRCOSTWD.solution_value)
print('TRCOSTRI value is:', TRCOSTRI.solution_value)
print('CWS_U_OUT value is:', CWS_U_OUT.solution_value)
print('CWS_N_IN value is:', CWS_N_IN.solution_value)
print('CWS_UR_OUT value is:', CWS_UR_OUT.solution_value)
print('CWS_R_IN value is:', CWS_R_IN.solution_value)
print('VARM value is:', VARM.solution_value)
print('VARI_R value is:', VARI_R.solution_value)
print('VARI_N value is:', VARI_N.solution_value)
print('VARC value is:', VARC.solution_value)
print('VARRM value is:', VARRM.solution_value)
print('REXC_WD value is:', REXC_WD.solution_value)
print('REXC_CD value is:', REXC_CD.solution_value)
print('REXC_WD_rem value is:', REXC_WD_rem.solution_value)
print('SCM value is:', SCM.solution_value)
print('SCR value is:', SCR.solution_value) 
print('TRCOSTUREM value is:', TRCOSTUREM.solution_value)
print('SCTCDU value is:', SCTCDU.solution_value)
print('HanW value is:', HanW.solution_value)

In [ ]:
##Dataframe for profit and costs results


header = ['Objective value','REV' , 'FOCC' , 'FOCI' , 'FOCRM','TRCOSTN',  'TRCOSTNW',  'TRCOSTRW', 'TRCOSTU', 'TRCOSTCD', 'TRCOSTWD','TRCOSTUREM', 'TRCOSTRI', 'CWS_U_OUT' , 'CWS_N_IN', 'CWS_UR_OUT', 'CWS_R_IN', 'VARM', 'VARI_R', 'VARI_N', 'VARC', 'VARRM', 'REXC_WD', 'REXC_CD', 'REXC_WD_rem', 'SCM', 'SCR','SCTCDU',  'HanW']
df4 = pd.DataFrame(columns = header)

df4.loc[len(df)-1, 'Objective value'] = model.objective_value
df4.loc[len(df)-1, 'FOCC'] = FOCC.solution_value
df4.loc[len(df)-1, 'FOCI'] = FOCI.solution_value
df4.loc[len(df)-1, 'FOCRM'] = FOCRM.solution_value
df4.loc[len(df)-1, 'REV'] = REV.solution_value
df4.loc[len(df)-1, 'TRCOSTN'] = TRCOSTN.solution_value
df4.loc[len(df)-1, 'TRCOSTNW'] = TRCOSTNW.solution_value
df4.loc[len(df)-1, 'TRCOSTRW'] = TRCOSTRW.solution_value
df4.loc[len(df)-1, 'TRCOSTU'] = TRCOSTU.solution_value
df4.loc[len(df)-1, 'TRCOSTCD'] = TRCOSTCD.solution_value
df4.loc[len(df)-1, 'TRCOSTWD'] = TRCOSTWD.solution_value
df4.loc[len(df)-1, 'TRCOSTUREM'] = TRCOSTUREM.solution_value
df4.loc[len(df)-1, 'TRCOSTRI'] = TRCOSTRI.solution_value
df4.loc[len(df)-1, 'CWS_U_OUT'] = CWS_U_OUT.solution_value  
df4.loc[len(df)-1, 'CWS_N_IN'] = CWS_N_IN.solution_value
df4.loc[len(df)-1, 'CWS_UR_OUT'] = CWS_UR_OUT.solution_value
df4.loc[len(df)-1, 'CWS_R_IN'] = CWS_R_IN.solution_value
df4.loc[len(df)-1, 'VARM'] = VARM.solution_value
df4.loc[len(df)-1, 'VARI_R'] = VARI_R.solution_value
df4.loc[len(df)-1, 'VARI_N'] = VARI_N.solution_value
df4.loc[len(df)-1, 'VARC'] = VARC.solution_value
df4.loc[len(df)-1, 'VARRM'] = VARRM.solution_value
df4.loc[len(df)-1, 'REXC_WD'] = REXC_WD.solution_value
df4.loc[len(df)-1, 'REXC_CD'] = REXC_CD.solution_value
df4.loc[len(df)-1, 'REXC_WD_rem'] = REXC_WD_rem.solution_value
df4.loc[len(df)-1, 'SCM'] = SCM.solution_value
df4.loc[len(df)-1, 'SCR'] = SCR.solution_value
df4.loc[len(df)-1, 'SCTCDU'] = SCTCDU.solution_value
df4.loc[len(df)-1, 'HanW'] = HanW.solution_value

In [ ]:
##Dataframe for demand per period

for T in range (0,61):
    DemNNew[T-1] = model.sum(Dn[T-1,W-1] for W in w)
for T in range (0,61):
    DemRNew[T-1] = model.sum(Dr[T-1,W-1] for W in w)
    
header = [ 'DemNAgg', 'DemRAgg']
df5 = pd.DataFrame(columns = header)
for T in range (0,61):
        df5.loc[len(df5), 'Period'] = T
        df5.loc[len(df5)-1, 'DemNAgg'] = DemNNew[T].solution_value
        df5.loc[len(df5)-1, 'DemRAgg'] = DemRNew[T].solution_value 
print(df5)

In [ ]:
##Excel export

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Solution_OEM_Reman_separate facilities_beta0.04.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name='Results_beta=0,04')
df4.to_excel(writer, sheet_name='Sum values')
df5.to_excel(writer, sheet_name="Demand")
# Close the Pandas Excel writer and output the Excel file.
writer.save()